In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from minigrid.wrappers import ImgObsWrapper
from minigrid_cnn import MiniGridFeaturesExtractor 
import torch as th

# --- 参数配置 ---
ENV_ID = "MiniGrid-Empty-8x8-v0"
TARGET_FEATURES_DIM = 128 

def validate_minigrid_cnn():
    # 1. 初始化环境
    env = gym.make(ENV_ID, render_mode=None)
    env = ImgObsWrapper(env)
    
    # 2. 检查观测空间形状
    obs_space = env.observation_space
    print(f"原始环境观测空间: {obs_space.shape}")
    
    # 【修正】：MiniGrid 原始输出确实是 (7, 7, 3)，这是正常的，不需要报错
    if obs_space.shape == (7, 7, 3):
        print("✅ 观测空间形状正确 (Height, Width, Channel)。")
    else:
        print(f"⚠️ 警告：观测空间形状非标准: {obs_space.shape}")

    # 3. 初始化定制的特征提取器
    # 这里会自动处理 (7, 7, 3) 的空间定义
    feature_extractor = MiniGridFeaturesExtractor(
        obs_space, 
        features_dim=TARGET_FEATURES_DIM
    )
    print(f"Features Extractor 初始化成功。")
    
    # 4. 关键验证：模拟 SB3 的处理流程
    obs_sample = obs_space.sample() # 获取一个 (7, 7, 3) 的样本
    
    # 转换为 Tensor 并增加 Batch 维度 -> (1, 7, 7, 3)
    obs_tensor = th.as_tensor(obs_sample[None]).float() 
    
    # 【关键修正】：手动模拟 SB3 的 VecTransposeImage 操作
    # 将 (Batch, H, W, C) 转换为 (Batch, C, H, W)
    obs_tensor = obs_tensor.permute(0, 3, 1, 2)
    
    print(f"送入 CNN 的张量形状 (模拟 SB3 转换后): {obs_tensor.shape}")

    # 运行 CNN
    with th.no_grad():
        features = feature_extractor(obs_tensor)

    # 5. 验证输出
    print(f"CNN 输出特征形状: {features.shape}")
    
    if features.shape == (1, TARGET_FEATURES_DIM):
        print("\n✅ **验证通过**：CNN 能够处理 MiniGrid 图像并输出正确维度的特征向量！")
        return True
    else:
        print(f"\n❌ **验证失败**：输出形状不匹配。")
        return False

if __name__ == "__main__":
    if validate_minigrid_cnn():
        print("\n---")
        print("太棒了！地基已经修补完成。请告诉我，我们是否可以开始编写核心的【Reward Shaping】代码？")

原始环境观测空间: (7, 7, 3)
✅ 观测空间形状正确 (Height, Width, Channel)。
Features Extractor 初始化成功。
送入 CNN 的张量形状 (模拟 SB3 转换后): torch.Size([1, 3, 7, 7])
CNN 输出特征形状: torch.Size([1, 128])

✅ **验证通过**：CNN 能够处理 MiniGrid 图像并输出正确维度的特征向量！

---
太棒了！地基已经修补完成。请告诉我，我们是否可以开始编写核心的【Reward Shaping】代码？
